In [1]:
from bdp_calcs import (
  parsear_tiempos, calc_sueno, calc_estimulantes, calc_actividad_luz_pantalla,
  calc_psicosocial, calc_medicacion_sustancias, calc_indices, calc_temporales
)
import pandas as pd

df = pd.read_csv("data/bdp_personal.csv")
df = parsear_tiempos(df)
df = calc_sueno(df)
df = calc_estimulantes(df)
df = calc_actividad_luz_pantalla(df)
df = calc_psicosocial(df)
df = calc_medicacion_sustancias(df)
df = calc_indices(df)
df = calc_temporales(df, ema_cols=["animo","claridad","estres","activacion"],
                     lagcols=["animo","claridad","estres","activacion"])

/home/fabo/BDP/ohlongday/bdp_calcs/utils.py:86: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dt = pd.to_datetime(combo.str.strip(), errors="coerce", dayfirst=dayfirst, infer_datetime_format=True)
/home/fabo/BDP/ohlongday/bdp_calcs/utils.py:86: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  dt = pd.to_datetime(combo.str.strip(), errors="coerce", dayfirst=dayfirst, infer_datetime_format=True)


In [2]:
from bdp_calcs.audit import core_audit
from bdp_calcs.drivers import *
from bdp_calcs.relations import *

In [3]:
import os, sys, argparse
import pandas as pd

In [4]:

input_csv ="data/bdp_personal.csv"
targets = "animo,claridad,estres,activacion".split(",")
output ="./outputs"
row_idx = -1

df = pd.read_csv(input_csv, encoding="utf-8")

In [5]:
#targets = [t.strip() for t in targets if t.strip()]
audit_df = core_audit(df, targets)
audit_path = output + "/core_audit.csv"
audit_df.to_csv(audit_path, index=False, encoding="utf-8")
print("Audit guardado en:", audit_path)
print(audit_df)

# Drivers del día para cada target
for t in targets:
    res = drivers_del_dia(df, t, top_k=3)
    txt = res.get("texto","")
    tab = res.get("tabla")
    narrativa = res.get("narrativa")
    print("\n[Drivers del día]", t, "→", txt, "\nInterpretación →", narrativa)
    if isinstance(tab, pd.DataFrame) and not tab.empty:
        tab.to_csv(os.path.join(output, f"{t}_drivers_del_dia.csv"), index=False, encoding="utf-8")


/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:1641: FutureWarning: 'alphas=None' is deprecated and will be removed in 1.9, at which point the default value will be set to 100. Set 'alphas=100' to silence this warning.
  warnings.warn(
/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:1641: FutureWarning: 'alphas=None' is deprecated and will be removed in 1.9, at which point the default value will be set to 100. Set 'alphas=100' to silence this warning.
  warnings.warn(
/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:1641: FutureWarning: 'alphas=None' is deprecated and will be removed in 1.9, at which point the default value will be set to 100. Set 'alphas=100' to silence this warning.
  warnings.warn(
/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:1641: Fut

Audit guardado en: ./outputs/core_audit.csv
       target  coverage       rel       ac1  mae_model  mae_naive  mae_ema  \
0       animo       1.0  0.500000  0.339450   1.279255       0.25      NaN   
1    claridad       1.0  0.333333  0.699031   0.591495       0.50      NaN   
2      estres       1.0  0.666667  0.598668   1.376231       1.00      NaN   
3  activacion       1.0  0.000000  0.134615   1.229001       0.75      NaN   

   gain_vs_best_baseline  actionability_pct  distinctiveness  n_samples  \
0              -4.117019                1.0              NaN         11   
1              -0.182990                1.0              NaN         11   
2              -0.376231                1.0              NaN         11   
3              -0.638668                1.0              NaN         11   

   n_splits  
0         2  
1         2  
2         2  
3         2  

[Drivers del día] animo → Animo: Δobs=0.0; Δpred≈-0.78 | ↑ meditacion_min, exposicion_sol_min, horas_sueno | ↓ aliment

/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [6]:
# 1) Wellbeing index
wb = wellbeing_index(df, targets=targets)
wb_path = os.path.join(output, "wellbeing_index_series.csv")
pd.DataFrame({"WB": wb}).to_csv(wb_path, index=False, encoding="utf-8")
print("WB guardado en:", wb_path)

# 2) Correlaciones (niveles y deltas)
cors = core_correlations(df, targets=targets)
cors["levels"].to_csv(os.path.join(output, "core_cor_levels.csv"), encoding="utf-8")
cors["deltas"].to_csv(os.path.join(output, "core_cor_deltas.csv"), encoding="utf-8")
print("Correlaciones listas: core_cor_levels.csv, core_cor_deltas.csv")

# 3) Influencia cruzada (lag-1)
infl = crosslag_influence(df, targets=targets)
if isinstance(infl, pd.DataFrame) and not infl.empty:
    infl.to_csv(os.path.join(output, "core_crosslag_influence.csv"), encoding="utf-8")
    print("Influencia cruzada guardada en: core_crosslag_influence.csv")
else:
    print("Influencia cruzada: insuficiente señal (necesita más filas).")

# 4) Agregación de drivers del día a lo largo de la historia
agg = aggregate_drivers_history(df, drivers_func=drivers_del_dia, targets=targets, top_k=3)
# Exportar por target
for t, ser in agg["avg_contrib"].items():
    if not ser.empty:
        ser.to_csv(os.path.join(output, f"{t}_avg_contrib.csv"), header=["avg_contrib"], encoding="utf-8")
for t, ser in agg["abs_contrib"].items():
    if not ser.empty:
        ser.to_csv(os.path.join(output, f"{t}_abs_contrib.csv"), header=["mean_abs_contrib"], encoding="utf-8")
for t, ser in agg["freq_sign"].items():
    if not ser.empty:
        ser.to_csv(os.path.join(output, f"{t}_freq_pos.csv"), header=["freq_positive"], encoding="utf-8")
print("Agregación de drivers exportada (avg/abs/freq).")

# 5) Sinergia de bienestar (qué drivers tienden a mejorar/empeorar el compuesto)
syn = wellbeing_synergy(agg)
if not syn.empty:
    syn.to_csv(os.path.join(output, "wellbeing_synergy.csv"), header=["wb_synergy"], encoding="utf-8")
    print("wellbeing_synergy.csv creado (features que más ayudan/perjudican WB).")
else:
    print("Sinergia de bienestar: aún sin datos suficientes.")

WB guardado en: ./outputs/wellbeing_index_series.csv
Correlaciones listas: core_cor_levels.csv, core_cor_deltas.csv
Influencia cruzada: insuficiente señal (necesita más filas).


/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but St

Agregación de drivers exportada (avg/abs/freq).
wellbeing_synergy.csv creado (features que más ayudan/perjudican WB).


In [11]:
#resumen = narrative_summary(df, targets)
resumen = descriptive_summary(df, targets,row_idx=4)
resumen

/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/miniconda3/envs/jupyter_env/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/fabo/BDP/ohlongday/bdp_calcs/relations.py:383: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass

'22-08-2025 (Fri): su bienestar compuesto está a la baja. Por targets: ánimo estable (Δ+0.00); claridad estable (Δ+0.00); estrés al alza (desfavorable) (Δ+1.00); activación estable (Δ+0.00). Motivos probables del día: Ánimo: apoyaron exposición al sol, respiración/meditación; restaron alimentación, horas de sueño. Claridad: apoyaron alimentación, horas de sueño; restaron exposición al sol, respiración/meditación. Estrés: apoyaron exposición al sol, respiración/meditación; restaron alimentación, horas de sueño. Activación: apoyaron exposición al sol, respiración/meditación; restaron alimentación, horas de sueño. Contexto: horas de sueño +0.00h, glicemia +32.00. Se recomienda: Favorece comidas balanceadas y horarios regulares para estabilizar glicemia. Mensaje para terapeuta: Explorar estresores recientes y estrategias de afrontamiento que ayudaron/ no ayudaron.'